# ハンズオン環境

https://colab.research.google.com/
へアクセスし、"uplopad"をクリックして本ノートブックをアップロードしてください。


-sandbox
# ライブイベント向け商品レコメンド - スタジアムアナリティクス
<img style='float: right' width='600px' src='https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-notif.png'>

**ユースケースの概要**
* スポーツのライブ中継では大量の顧客データが生成されます。より良いファン体験を実現するのに活用できます。ファンとの関わりは、売上や顧客維持率の向上に直結します。このノートブックでは、ファンの購買履歴とスタジアムの座席の位置情報をもとにパーソナライズされた割引オファーを作成し、イベント中の追加販売を促進し、より個人的な体験を実現する方法を紹介します。
*目標：ファンへキャンペーンのオファーのプッシュ通知を送信します。
 
**ソリューションのビジネスインパクト**
* **ファン・エンゲージメントと顧客維持：** イベントに参加している間、より良い経験をしているファンは、将来的に別のイベントのために戻ってくる可能性が高くなります。
* **収益の増加：**ファンに パーソナライズされた割引キャンペーンを送信すると、イベント中にスタジアムの中の店からの購入が増えるのでお店(ベンダー)の収益の増加にも繋がります。

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://www.google-analytics.com/collect?v=1&gtm=GTM-NKQ8TT7&tid=UA-163989034-1&cid=555&aip=1&t=event&ec=field_demos&ea=display&dp=%2F42_field_demos%2Fmedia%2Fproduct_recommender_stadium%2Fnotebook&dt=MEDIA_USE_CASE">
<!-- [metadata={"description":"Product recommendation for live events. This demo shows how to create a personalized discount for a fan based on their purchasing history and location of where they sit in a stadium to drive additional sales during the game and create a more individualized experience.",
 "authors":["dan.morris@databricks.com"],
  "db_resources":{},
  "search_tags":{"vertical": "media", "step": "Data Engineering", "components": ["sparkml", "mlflow", "als", "recommender"]},
                 "canonicalUrl": {"AWS": "", "Azure": "", "GCP": ""}}] -->

# 設定と準備

## Sparkの準備

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 39.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=e5e6c6892d26a759fe61d9fcce38ae0e812b4bacd76356426074ce71d6e7ef34
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("IAB") \
    .getOrCreate()


In [3]:
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

## MLflowの準備

In [4]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 3.8 MB/s 
     |████████████████████████████████| 209 kB 57.2 MB/s 
     |████████████████████████████████| 147 kB 58.9 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 182 kB 55.1 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 140 kB 68.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139103 sha256=21cc12f832de373481f100c7836540b7e4dbd5c0bb8a0ebeabbf60d1d8670b5c
  Stored in directory: /root/.cache/pip/wheels/3f/73/87/c1e4b2145eb6049bb6c9aaf7ea1e38302b77ca219b6fef5d5c


## ダミデータ作成のdbldatagenのインストール

In [5]:
%pip install git+https://github.com/databrickslabs/dbldatagen faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/databrickslabs/dbldatagen to /tmp/pip-req-build-r_ffsdhn
  Running command git clone -q https://github.com/databrickslabs/dbldatagen /tmp/pip-req-build-r_ffsdhn
     |████████████████████████████████| 1.6 MB 4.3 MB/s 
  Created wheel for dbldatagen: filename=dbldatagen-0.2.1-py3-none-any.whl size=68644 sha256=7602bbb105e7ed2d1b29ee956c1437d3fb8d5a5d7a684564094bd98b30b3c9d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-tuu8zise/wheels/fa/2f/29/fd337c2f0a1da95c1069d7f4898c3ff5a0ea697cb5d31f319c
Successfully built dbldatagen


## データの準備

以下リンクからデータのZipファイルをダウンロードしてGoogle colabの/content/の直下にアップロードしてください。

https://github.com/yulan-yan/Product_recommendation_stadium/blob/main/stadium_recommender.zip


In [6]:
!unzip /content/stadium_recommender.zip

Archive:  /content/stadium_recommender.zip
   creating: stadium_recommender/
  inflating: stadium_recommender/.DS_Store  
  inflating: __MACOSX/stadium_recommender/._.DS_Store  
   creating: stadium_recommender/data/
  inflating: stadium_recommender/data/.DS_Store  
  inflating: __MACOSX/stadium_recommender/data/._.DS_Store  
  inflating: stadium_recommender/data/vendors.csv  
  inflating: __MACOSX/stadium_recommender/data/._vendors.csv  
  inflating: stadium_recommender/data/games.csv  
  inflating: __MACOSX/stadium_recommender/data/._games.csv  
  inflating: stadium_recommender/data/purchase_history.csv  
  inflating: __MACOSX/stadium_recommender/data/._purchase_history.csv  


 ## パスなど変数の設定

In [7]:
data_path = "/content/stadium_recommender"
dbName = "yulanDB"
current_user = "Yulan"

## Databaseの準備

In [8]:
# オプション：Googleドライブのマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
spark.sql("drop database if exists yulanDB")
spark.sql("create database if not exists yulanDB")
spark.sql("use yulanDB")

DataFrame[]

# レコメンデーションシステムの構築のワークフロー

## Step 1： データを読込・加工したらraw tablesに書き込む

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-1.png" width="1000px">




### VendorとItemのリスト（stadium_vendors）

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-vendors.png" width="400px" style="float: right" />

ベンダーリストとアイテムのリストを[https://www.fordfield.com/restaurant-partners](Ford Field)へCSVファイルとしてアップロードしています。
 
PandasとSparkを使えば、これらのデータを簡単に取り込み、データ型を調整することができます。

In [10]:
print("Vendors Item type data generation...")
import pandas as pd
from pyspark.sql.functions import col

pandasDF = pd.read_csv(f"{data_path}/data/vendors.csv")
df_items = spark.createDataFrame(pandasDF, ['vendor_id', 'vendor_location_number', 'vendor_name', 'vendor_scope', 'section', 'item_id', 'item_type', 'item', 'price', 'error'])
for c in "vendor_id", "vendor_location_number", "section", "item_id", "price":
  df_items = df_items.withColumn(c, col(c).cast('int'))

spark.sql("drop table if exists stadium_vendors")
df_items.write.saveAsTable("stadium_vendors")

Vendors Item type data generation...



### チケットの売上データ（Tickets sales）

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-pricing.png" width="400px" style="float: right" />

チケットの売上データもraw tableに書き込みます。

In [11]:
print("Ticket sales data generation...")
import dbldatagen as dg
from pyspark.sql.types import *
from pyspark.sql import functions as F
from faker import Faker
fake = Faker()

fake_name = F.udf(fake.name)
fake_phone = F.udf(fake.phone_number)

data_rows = 70000
df_spec = (dg.DataGenerator(spark, name="ticket_sales", rows=data_rows, partitions=4)
                            .withIdOutput()
                            .withColumn("game_id", IntegerType(), values=['3'])
                            .withColumn("ticket_price", IntegerType(), expr="floor(rand() * 350)")
                            .withColumn("gate_entrance", StringType(), values=['a', 'b', 'c', 'd', 'e', ], random=True, weights=[9, 1, 1, 2, 2])
                            .withColumn("section_number", IntegerType(), minValue=100, maxValue=347, random=True)
                            .withColumn("row_number", IntegerType(), minValue=1, maxValue=35 , random=True)
                            .withColumn("seat_number", IntegerType(), minValue=1, maxValue=30, random=True)
                            .withColumn("ticket_type", StringType(), values=['single_game', 'packaged_game', 'season_holder'], random=True, weights=[7, 2, 1]))
df_tickets = df_spec.build().withColumnRenamed("id", "customer_id")
df_tickets = df_tickets.withColumn("customer_name", fake_name())
df_tickets = df_tickets.withColumn("phone_number", fake_phone())

spark.sql("drop table if exists ticket_sales")
df_tickets.write.saveAsTable("ticket_sales")

Ticket sales data generation...


### イベントのスケジュール（Games）

試合のイベントデータも同じ方法で取り込みます。

In [12]:
print("Game data generation...")
from pyspark.sql.functions import col, to_date
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY")

schemaDDL = "game_id int, date_time string, date string, location string, home_team string, away_team string, error string, game_date string"
df_games = spark.read.csv(f"{data_path}/data/games.csv", sep=',', schema=schemaDDL)
df_games = df_games.withColumn("game_date", to_date(col("game_date"),'yyyy-MM-dd'))

spark.sql("drop table if exists games")
df_games.write.saveAsTable("games")

Game data generation...


### Points of Saleデータ

POSデータも取り込みます。

In [13]:
print("Point of sales generation...")
data_rows = 1000000
df_spec = (dg.DataGenerator(spark, name="point_of_sale", rows=data_rows, partitions=4)
                            .withIdOutput()
                            .withColumn("game", IntegerType(), minValue=1, maxValue=97 , random=True)
                            .withColumn("item_purchased", IntegerType(), minValue=1, maxValue=364, random=True)
                            .withColumn("customer", IntegerType(), minValue=1, maxValue=1000, random=True))
                            
df_pos = df_spec.build().withColumnRenamed("id", "order_id")

# tableにデータを書き込みます。
spark.sql("drop table if exists point_of_sale")
df_pos.write.saveAsTable("point_of_sale")

Point of sales generation...


### 購入履歴(purchase_history)を取り込みます。


In [14]:
print("Purchase History to evaluate model efficiency after real game...")
import pandas as pd
pandasDF = pd.read_csv(f"{data_path}/data/purchase_history.csv")
df_spec = spark.createDataFrame(pandasDF, ['item_id', 'game_id', 'customer_id', 'recommended_item_purchased'])

spark.sql("drop table if exists purchase_history")
df_spec.write.saveAsTable("purchase_history")

Purchase History to evaluate model efficiency after real game...


## Step 2: Silver tableを作成する

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-2.png" width="1000px">

In [15]:
silver_sales_df = spark.sql('''
  SELECT * FROM ticket_sales t 
  JOIN point_of_sale p ON t.customer_id = p.customer
  JOIN stadium_vendors s ON p.item_purchased = s.item_id AND t.game_id = p.game;  
''')

silver_sales_df.createTempView("silver_sales")
silver_sales_df.show()

+-----------+-------+------------+-------------+--------------+----------+-----------+-------------+-----------------+--------------------+--------+----+--------------+--------+---------+----------------------+--------------------+------------+-------+-------+---------+--------------------+-----+-----+
|customer_id|game_id|ticket_price|gate_entrance|section_number|row_number|seat_number|  ticket_type|    customer_name|        phone_number|order_id|game|item_purchased|customer|vendor_id|vendor_location_number|         vendor_name|vendor_scope|section|item_id|item_type|                item|price|error|
+-----------+-------+------------+-------------+--------------+----------+-----------+-------------+-----------------+--------------------+--------+----+--------------+--------+---------+----------------------+--------------------+------------+-------+-------+---------+--------------------+-----+-----+
|        409|      3|          23|            a|           131|         3|         24|pa

最も購入されたアイテムのトップ 10

In [16]:
spark.sql('''
 SELECT item_id, count(item_id) AS item_purchases FROM silver_sales GROUP BY item_id order by item_purchases desc limit 10
''').show()

+-------+--------------+
|item_id|item_purchases|
+-------+--------------+
|    344|            44|
|     94|            43|
|    187|            43|
|    170|            43|
|    110|            43|
|     86|            42|
|     56|            41|
|    124|            41|
|    275|            41|
|    202|            40|
+-------+--------------+



## Step 3: リコメンデーション・モデルの構築

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-3.png" width="1000px">

### MLFlowを用いたALSレコメンダーの構築
 

Mlflowは、モデル自身を含むすべての実験メトリクスの追跡に使用されています。

構築したモデルを再利用して、最終的なレコメンデーションを取得し、パーソナルなオファーをプッシュ通知で送ることができるようになります。

In [17]:
import mlflow
from pyspark.ml.recommendation import ALS

with mlflow.start_run() as run:
  df = spark.sql("select customer_id, item_id, count(item_id) as item_purchases from silver_sales group by customer_id, item_id")
  # ALSを使用してトレーニングデータからレコメンデーションモデルを構築する。
  # コールドスタート戦略を'drop'に設定して、NaN評価メトリクスにならないようにすることに注意してください。
  # 評価マトリックスは別の情報源から導出されます (つまり、他の情報から推測されます)。より良い結果を得るために、implicitPrefs を true に設定します。
  als = ALS(rank=3, userCol="customer_id", itemCol="item_id", ratingCol="item_purchases", implicitPrefs=True, seed=0, coldStartStrategy="nan")
  
  num_cores = sc.defaultParallelism
  als.setNumBlocks(num_cores)
  
  model = als.fit(df)
  #　別のセルからこの実行の他の数値を追加する必要があるため、実行IDを取得しましょう。
  run_id = run.info.run_id


## Step 4: モデルでレコメンデーションリストを作成する

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-4.png" width="1000px">


In [18]:
# 顧客毎の上位10件のレコメンデーションアイテムを取得する
recommendations = model.recommendForAllUsers(10)

spark.sql("drop table if exists silver_recommendations")
recommendations.createOrReplaceTempView("silver_recommendations")
recommendations.show()

+-----------+--------------------+
|customer_id|     recommendations|
+-----------+--------------------+
|          1|[{110, 0.06207476...|
|          2|[{225, 0.21452291...|
|          3|[{275, 0.09176123...|
|          4|[{110, 0.10726592...|
|          5|[{344, 0.10935707...|
|          6|[{344, 0.07213435...|
|          7|[{124, 0.18917444...|
|          8|[{124, 0.32242697...|
|          9|[{137, 0.25173628...|
|         10|[{305, 0.22389345...|
|         11|[{225, 0.17785867...|
|         12|[{33, 0.16146883}...|
|         13|[{33, 0.1658469},...|
|         14|[{110, 0.09178765...|
|         15|[{124, 0.23991564...|
|         16|[{170, 0.09936026...|
|         17|[{124, 0.15195665...|
|         18|[{33, 0.12710817}...|
|         19|[{33, 0.15763727}...|
|         20|[{124, 0.21426772...|
+-----------+--------------------+
only showing top 20 rows



## Step 5: レコメンデーション結果のフィルタリングと再ランク付け


顧客毎のレコメンドを展開し、アイテムや顧客の詳細を取得します。

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-5.png" width="1000px">

In [19]:
gold_recommendations_df = spark.sql('''
SELECT ci.*, vendor_id, vendor_location_number, vendor_name, vendor_scope, section, item_type, item, price, error
FROM stadium_vendors v
JOIN (SELECT customer_id, items.*
FROM   (SELECT customer_id, explode(recommendations) AS items
FROM   silver_recommendations) a) ci
ON v.item_id = ci.item_id
''') 

spark.sql("drop table if exists gold_recommendations")
gold_recommendations_df.createTempView("gold_recommendations")
spark.sql('select * from gold_recommendations').show()

+-----------+-------+-----------+---------+----------------------+--------------------+------------+-------+-------------+--------------------+-----+-----+
|customer_id|item_id|     rating|vendor_id|vendor_location_number|         vendor_name|vendor_scope|section|    item_type|                item|price|error|
+-----------+-------+-----------+---------+----------------------+--------------------+------------+-------+-------------+--------------------+-----+-----+
|          1|    110| 0.06207476|       17|                     1|  Streats of Detroit|        Food|    109|       Nachos|        Nacho Grande|   14|  NaN|
|          1|    137|0.060412034|       17|                     3|  Streats of Detroit|        Food|    124|        Snack|         Cheese Cups|   10|  NaN|
|          1|    170|0.055190045|       17|                     6|  Streats of Detroit|        Food|    137|        Snack|         Cheese Cups|   10|  NaN|
|          1|    225|0.051179606|       17|                    1

### 顧客毎のベストアイテムを取得します。

アイテムのパーソナライゼドレコメンドを行う時に、近い距離で購入できるアイテムを探したいです。そのために、お客様の席からお店の場所までの距離を計算します。

In [20]:
sections_recommendations_df = spark.sql("""
SELECT r.customer_id, item_id, rating, section, section_number, abs(section-section_number) as distance
FROM gold_recommendations r
JOIN ticket_sales s on s.customer_id = r.customer_id
""")
sections_recommendations_df.createTempView("sections_recommendations")
       
spark.sql("select * from sections_recommendations").show()

+-----------+-------+-----------+-------+--------------+--------+
|customer_id|item_id|     rating|section|section_number|distance|
+-----------+-------+-----------+-------+--------------+--------+
|          1|    110| 0.06207476|    109|           127|      18|
|          1|    137|0.060412034|    124|           127|       3|
|          1|    170|0.055190045|    137|           127|      10|
|          1|    225|0.051179606|    229|           127|     102|
|          1|    234|0.049891207|    244|           127|     117|
|          1|    187|0.049530078|    223|           127|      96|
|          1|    156|  0.0489764|    130|           127|       3|
|          1|     94| 0.04766779|    103|           127|      24|
|          1|    147|0.046597462|    126|           127|       1|
|          1|    348|0.046327006|    214|           127|      87|
|          2|    225| 0.21452291|    229|           179|      50|
|          2|    170| 0.20312713|    137|           179|      42|
|         

In [21]:
final_recommendations_df = spark.sql("""
SELECT * FROM (
  SELECT *, RANK() OVER (PARTITION BY customer_id ORDER BY distance ASC) AS rnk FROM sections_recommendations
  ) WHERE rnk = 1
""")

final_recommendations_df.createTempView("final_recommendations")
spark.sql("SELECT * FROM final_recommendations").show()

+-----------+-------+-----------+-------+--------------+--------+---+
|customer_id|item_id|     rating|section|section_number|distance|rnk|
+-----------+-------+-----------+-------+--------------+--------+---+
|          1|    147|0.046597462|    126|           127|       1|  1|
|          2|    341| 0.17146161|    214|           179|      35|  1|
|          2|    348| 0.17122616|    214|           179|      35|  1|
|          3|    333| 0.08083317|    206|           204|       2|  1|
|          4|    137| 0.10688759|    124|           117|       7|  1|
|          5|    202|  0.0948009|    227|           229|       2|  1|
|          6|    277| 0.06821223|    242|           277|      35|  1|
|          7|    239|  0.1689277|    244|           343|      99|  1|
|          8|    239| 0.29360628|    244|           280|      36|  1|
|          9|    348| 0.18603727|    214|           186|      28|  1|
|         10|    343| 0.19850445|    214|           221|       7|  1|
|         10|    350

## Step 6: ファンにキャンペーンのオファーの送信

これで、全顧客に送るべきアイテムが揃いました！通知をテストしてみましょう。

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-6.png" width="1000px">

In [22]:
import IPython
from IPython.display import display, HTML

def send_push_notification(title, subject, phone_number):
  #ノートブックを実行しているユーザーでメッセージを調整してください。リアルでは、顧客の詳細を含むテーブルから情報を取るべきです。
  first_name = "Yulan"
  import re
  subject = re.sub("\n", "<br/>", subject) 
  subject = re.sub("こんにちは！([0-9]*)", "こんにちは！"+first_name, subject) 
  subject = re.sub("大好きな([0-9]*)", "大好きなピザ", subject) 
  subject = re.sub("([0-9]*)で販売されている", "Goal Post Grill & Pizzeriaで販売されている", subject) 


  htm = HTML(f"""<div style="border-radius: 10px; background-color: #adeaff; padding: 10px; width: 400px; box-shadow: 2px 2px 2px #F7f7f7; margin-bottom: 3px">
        <div style="padding-bottom: 5px"><img style="width:20px; margin-bottom: -3px" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/bell.png"/> <strong>{title}</strong></div>
        {subject}
        </div>""")
  display(htm)

In [23]:
recommendation = spark.sql("select * from final_recommendations where customer_id = 1").collect()[0]

title = "あなたにお得なキャンペーンオファー！"
subject= f"こんにちは！{recommendation['customer_id']}さん! 大好きな{recommendation['item_id']}を特別価格でご提供いたします。 {recommendation['section']}で販売されているので、チェックしてみてくださいね!" 

user_cell_phone = "<Put your real cell phone here to test the notification!>"
 
send_push_notification(title, subject, user_cell_phone)

In [24]:
print(recommendation)

Row(customer_id=1, item_id=147, rating=0.04659746214747429, section=126, section_number=127, distance=1, rnk=1)


## Step 7: イベントキャンペーンとメトリクスのトラッキング

イベント中は、すべての指標をリアルタイムで取得し、イベントのKPIをモニターするダッシュボードを構築することができます。
 
ゲーム終了後、プロモーションの成功率を確認し、次のゲームに必要な調整を行うこともできます。

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/media/resources/images/product_reco_stadium-7.png" width="1000px">

In [25]:

## visualisation: Bar, Series groupings: item_purchased, Values: count, Aggregation: SUM

visual_df = spark.sql("""
select count(*) as count, item_purchased from (
  SELECT *, CASE WHEN recommended_item_purchased = 1 THEN 'Yes' ELSE 'No' END as item_purchased
  FROM final_recommendations r
  LEFT JOIN ticket_sales s USING(customer_id)
  LEFT JOIN purchase_history p USING(game_id, customer_id)) group by item_purchased
""")
visual_df.show()

+-----+--------------+
|count|item_purchased|
+-----+--------------+
|  479|            No|
|  760|           Yes|
+-----+--------------+

